# Tutorial 6: HuggingFace Client

In this tutorial, we'll explore how to use the `HuggingFaceClient`.

## Installation

If you don't have `RadPrompter` installed, you can install it using pip:

```bash
pip install radprompter
```

## Prompt

As always, we start by importing the `Prompt` class and creating a prompt object from a TOML file:

In [1]:
from radprompter import Prompt

prompt = Prompt('06_HuggingFace-Client.toml')
prompt

## Client and Engine

We'll use the new `HuggingFaceClient`. This model accepts the model and tokenizer from HuggingFace. This allows for all customizations on the model including quantization.

Before running the next cell, make sure you have the following libraries installed:

```bash
pip install torch transformers flash_attn SentencePiece accelerate bitsandbytes
```

In [2]:
import os
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name = "microsoft/Phi-3-mini-128k-instruct"

quantization_conf = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_conf, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from radprompter import RadPrompter, HuggingFaceClient

client = HuggingFaceClient(
    hf_model = model,
    hf_tokenizer = tokenizer,
    temperature = 0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_6.csv",
    concurrency=1,
    hide_blocks=False,
)

And we run it on our sample reports:

In [4]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

Processing items:   0%|          | 0/3 [00:00<?, ?it/s]/home/bkhosra/.conda/envs/RP/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/bkhosra/.conda/envs/RP/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/bkhosra/.conda/envs/RP/lib/python3.11/site-packages/transformers/generation/utils.py:1637: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected beha

The engine will process each report and saves the results to `output_tutorial_6.csv`.

In [5]:
import pandas as pd

df = pd.read_csv("output_tutorial_6.csv", index_col='index')
df

,default_response_0,default_response_1,report,file_name
index,,,,
0,\n<initial_answer>\nPresent\n</initial_answer>...,"{\n ""answer"": ""Present""\n}",Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt
1,\n<initial_answer>\nPresent\n</initial_answer>...,"{\n ""answer"": ""Present""\n}",Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
2,\n<initial_answer>\nAbsent\n</initial_answer>\...,"{\n ""answer"": ""Absent""\n}",Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt


Finally, we save the log:

In [6]:
engine.save_log("log_tutorial_6.log")

with open("log_tutorial_6.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.1.4
Model: Phi3ForCausalLM
Prompt TOML: /mnt/NAS3/homes/bkhosra/RadPrompter/tutorials/06_HuggingFace-Client/06_HuggingFace-Client.toml
Prompt Version: 0.1
Prompt Hash: 83cd2cee67b7e10589df0b8f9e9fd2f2
Concurrency Factor: 1
Start Time: 2024-05-20 12:18:22
End Time: 2024-05-20 12:18:38
Duration: 16.0
Number of Items: 3
Average Processing Time: 5.333333333333333


-------------------- *** - Prompt Content - *** --------------------
[METADATA]
version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]
system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured. Here is the report:
<report>
{{report}}
</report>
"""

user_prompt_cot = """
I want you to extract the following data element from the report: 'Pulmonary Embolism'
Here are yo

The `HuggingFaceClient` can be useful for beginning to explore RadPrompter's capabilities, but it is certainly **not the best option**. We highly advise using the `vLLMClient` or `OllamaClient`, as they support concurrency and are more stable for batch document processing.